# 02 - Controlador PID de Velocidad

Este notebook muestra:
- Por qué controlar la velocidad requiere un lazo de retroalimentación.
- Cómo afecta cada término del PID (*Proporcional, Integral, Derivativo*).
- Cómo filtrar la estimación de velocidad.
- Cómo ajustar los parámetros con una simulación interactiva.


In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("src")

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

In [3]:
# Supón que importaste desde src/motorcontrol:
from motorcontrol.rotary_motor import RotaryMotor, trapezoidal_profile
from motorcontrol.encoder import Encoder
from motorcontrol.filters import ExponentialFilter
from motorcontrol.speed_controller import SpeedControllerPID, SpeedControllerLogger, plot_speed_pid

def run_speed_pid_simulation(kp, ki, kd):
    # Motor físico
    motor = RotaryMotor(
        inertia=2.0,
        friction_viscous=0.1,
        max_torque=12.0,
        friction_coulomb=0.3
    )

    encoder = Encoder(motor, noise_std_deg=0.2)
    velocity_filter = ExponentialFilter(alpha=0.2)

    pid = SpeedControllerPID(
        motor, encoder, velocity_filter,
        kp=kp, ki=ki, kd=kd,
        integral_limit=50.0,
        deadband=0.5
    )

    vel_ref_signal = trapezoidal_profile(ramp_steps=200, hold_steps=300, peak_value=20.0)

    logger = SpeedControllerLogger()

    dt = 0.01

    for step, vel_ref in enumerate(vel_ref_signal):
        status = pid.update(vel_ref, dt)
        motor.update(dt)
        logger.log(step*dt, vel_ref, status, motor.get_velocity_deg_s())

    plot_speed_pid(logger)
    
interact(
    run_speed_pid_simulation,
    kp=widgets.FloatSlider(value=0.06, min=0.0, max=1.0, step=0.02),
    ki=widgets.FloatSlider(value=0.02, min=0.0, max=0.5, step=0.01),
    kd=widgets.FloatSlider(value=0.00, min=0.0, max=0.2, step=0.01)
);


interactive(children=(FloatSlider(value=0.06, description='kp', max=1.0, step=0.02), FloatSlider(value=0.02, d…